- Generate test data
- Use NN to predict test data
- Validate NN methodology for prediction

In [2]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time 
import copy
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Input
from keras.models import Model

from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l1
from keras.models import Sequential
from keras.models import load_model

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)


Using TensorFlow backend.


In [3]:
# create a data set, sin wave plus trend plus random noise
nobs = 3000
x = np.linspace(0, 3*np.pi, num=nobs)
y = -np.cos(x) + x/(3*np.pi) + np.random.normal(0, 0.25, nobs)

x2 = np.linspace(0, 15*np.pi, num=nobs)
y1 = -np.sin(x2) + x2/(3*np.pi) + np.random.normal(0, 0.25, nobs)
y2 = -np.cos(x2) + x2/(3*np.pi) + np.random.normal(0, 0.25, nobs)


In [4]:

def mychart(*args):

    # pass some 2d n x 1 arrays, x, y, z
    
    # 1st array is independent vars
    # reshape to 1 dimensional array
    x = args[0].reshape(-1)
    
    # following are dependent vars plotted on y axis
    data = []
    for i in range(1, len(args)):
        data.append(Scatter(x=x,
                            y=args[i].reshape(-1),
                            mode = 'markers',
                            marker = dict(
                                size = 2,     
                            )
                           )
                   )

    layout = Layout(
        yaxis=dict(
            autorange=True))
    
    fig = Figure(data=data, layout=layout)
    
    return iplot(fig) # png to save notebook w/static image   
    
mychart(x2,y1,y2)

In [6]:
wdata = pd.DataFrame(np.array([y1,y2,y2]).transpose())
wdata[2] = wdata[2].shift(-1)
wdata = wdata.dropna(axis=0, how='any') 
wdata


,0,1,2
0,-0.190382,-1.283555,-1.147116
1,-0.087386,-1.147116,-1.724930
2,-0.151574,-1.724930,-0.503065
3,0.056968,-0.503065,-1.037448
4,-0.513047,-1.037448,-1.126757
5,-0.221079,-1.126757,-0.926448
6,0.335168,-0.926448,-0.826336
7,-0.228015,-0.826336,-1.050125
8,-0.305916,-1.050125,-0.784590
9,-0.004453,-0.784590,-1.060950


In [7]:
allcols = list(wdata.columns)
ncols = len(allcols)

responses = allcols[-1:]
nresponses = len(responses)

predictors = allcols[:ncols-nresponses]
npredictors = len(predictors)
# create inputs for NN
# 52 x 30 weekly changes
# 4-week change 4 weeks hence
# portfolio return will be mean(top6, -bot6)/4 (each week hold put 1/4 into portfolio and hold for 4 weeks)
# remove NAs
# predict 4-week change based on preceding 52 weeks


In [8]:
wdata[responses]

,2
0,-1.147116
1,-1.724930
2,-0.503065
3,-1.037448
4,-1.126757
5,-0.926448
6,-0.826336
7,-1.050125
8,-0.784590
9,-1.060950


In [9]:
wdata[predictors]

,0,1
0,-0.190382,-1.283555
1,-0.087386,-1.147116
2,-0.151574,-1.724930
3,0.056968,-0.503065
4,-0.513047,-1.037448
5,-0.221079,-1.126757
6,0.335168,-0.926448
7,-0.228015,-0.826336
8,-0.305916,-1.050125
9,-0.004453,-0.784590


In [13]:
wdata[responses].describe()


,2
count,2999.000000
mean,2.505872
std,1.627888
min,-1.724930
25%,1.230987
50%,2.547225
75%,3.765754
max,6.479638


In [14]:
wdata[predictors].describe()

,0,1
count,2999.000000,2999.000000
mean,2.461674,2.503521
std,1.627171,1.628268
min,-1.475858,-1.724930
25%,1.243627,1.228220
50%,2.496226,2.546952
75%,3.740363,3.763359
max,6.032254,6.479638


In [16]:
lookback = 12 # use 12 weeks of data to forecast
lead = 1 # forecast 1 weeks ahead
# first Y to predict is row 13 using 0:11
Y = wdata.values[lookback+lead-1:, -nresponses:]
numrows = Y.shape[0]

X_raw = wdata.values[:,:npredictors]

# each input will have cols 12 * npredictors
X = np.zeros([numrows, lookback * npredictors])
for row in range(numrows):
    prev12 = []
    for i in range(lookback):
        prev12.append(X_raw[row + i])
    X[row] = np.hstack(prev12)

print(X.shape)
print(Y.shape)

(2987, 24)
(2987, 1)


In [17]:
# double check this vs. above, predicting last Y using 12 Xs ending 4 prior
print (Y[-1])
print("---")
print (X[-1])

[5.76746505]
---
[4.50319022 5.94767809 4.78770956 6.18001863 5.01749194 6.11239196
 5.22072504 5.82823905 4.87469204 6.31722396 5.29489825 6.31628247
 5.24365281 5.59440719 4.4485531  5.78529454 4.79007533 6.22506512
 4.7009715  5.47242645 5.4351966  6.08539343 4.82458703 5.80372513]


In [18]:
OUTPUT_DIM = len(responses) # 1
OUTPUT_DIM

1

In [19]:

INPUT_DIM = X.shape[1] # 24
OUTPUT_DIM = len(responses) # 1

def build_model(n_hidden_layers = 2,
                hidden_layer_size = 32,
                reg_penalty = 0.0001,
                dropout = 0.333,
                verbose=True):

    main_input = Input(shape=(INPUT_DIM,), 
                       dtype='float32', 
                       name='main_input')
    lastlayer=main_input

    for i in range(n_hidden_layers):
        if verbose:
            print("layer %d size %d, reg_penalty %.8f, dropout %.3f" % (i, hidden_layer_size, reg_penalty, dropout))
        lastlayer = Dense(units = hidden_layer_size, 
                          activation = 'relu',
                          kernel_initializer = keras.initializers.glorot_uniform(),
                          kernel_regularizer=keras.regularizers.l1(reg_penalty),
                          name = "Dense%02d" % i)(lastlayer)

        if dropout:
            lastlayer = Dropout(dropout, name = "Dropout%02d" % i)(lastlayer)
    
    outputs = []
    for i in range(OUTPUT_DIM):
        # OUTPUT_DIM outputs
        output01 = Dense(1,
                         activation='linear', 
                         name='output%02d' % i)(lastlayer)
        outputs.append(output01)
    
    model = Model(inputs=[main_input], outputs=outputs)
    if verbose:
        print(model.summary())
    model.compile(loss="mse", optimizer="rmsprop", loss_weights=[1.]*OUTPUT_DIM)
    return model


In [20]:
# run an experiment with walk-forward cross-validation

EPOCHS = 500
#VAL_SPLIT = 0.2
BATCH_SIZE = 128

def run_experiment (n_hidden_layers = 2,
                    hidden_layer_size = 8,
                    reg_penalty = 0.001,
                    dropout = 0.25,
                    epochs = EPOCHS
                   ):

    start = time.time()

    # generate k-folds
    n_splits = 5
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)

    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))
    
    avg_bests = []

    print("%s Build model" % (time.strftime("%H:%M:%S")))
    model = build_model(n_hidden_layers = n_hidden_layers,
                        hidden_layer_size = hidden_layer_size,
                        reg_penalty = reg_penalty,
                        dropout = dropout)
    print("Compile time : %s" % str(time.time() - start))
    print("Starting to train : %s" % (time.strftime("%H:%M:%S")))
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        responses = []
        for i in range(OUTPUT_DIM):
            responses.append(Y_fit[:,i])
        # train for epochs
        for epoch in range(epochs):
            fit = model.fit(
                X_fit,
                responses,
                batch_size=BATCH_SIZE,
                #validation_split=VAL_SPLIT,
                epochs=1,
                verbose=0)
            
            train_loss = fit.history['loss'][-1]
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = np.array(model.predict(X_xval))
            y_xval_pred = y_xval_pred.reshape(Y_xval.T.shape)
            y_xval_pred = y_xval_pred.T
            mse_list = []
            for i in range(len(responses)):
                mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred[:,i]))
            xval_score = np.mean(np.array(mse_list))            
            
            losses.append(train_loss)
            scores.append(xval_score)
            models.append(copy.copy(model))

            bestloss_index = np.argmin(scores)
            bestloss_value = scores[bestloss_index]

            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()            
            
            # stop if loss rises by 20% from best
            if xval_score / bestloss_value > 1.2:
                print("Stopping early" )
                break

        # choose model with lowest xval loss
        print("")
        print ("%s Best Xval loss epoch %d, value %f" % (time.strftime("%H:%M:%S"), bestloss_index, bestloss_value))
        avg_bests.append(bestloss_value)
        model = models[bestloss_index]
    
    print ("Last Xval loss %f" % (bestloss_value))
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [41]:
run_experiment()


00:17:59 Generate splits [598, 1196, 1793, 2390, 2987]
00:17:59 Build model
layer 0 size 8, reg_penalty 0.00100000, dropout 0.250
layer 1 size 8, reg_penalty 0.00100000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 8)                 200       
_________________________________________________________________
Dropout00 (Dropout)          (None, 8)                 0         
_________________________________________________________________
Dense01 (Dense)              (None, 8)                 72        
_________________________________________________________________
Dropout01 (Dropout)          (None, 8)                 0         
_________________________________________________________________
output00 (Dense)        

(0.7574146287272577, <keras.engine.training.Model at 0x7fa379fb80b8>)

In [42]:
# run a lot of experiments in big xval loop
# make predictions
# pick best hyperparameters

MODELPREFIX = "FFNN"

n_hiddens = [1, 2, 3]
layer_sizes = [4, 8, 16]
reg_penalties = [0.0, 0.001, 0.01]
dropouts = [0.25]

hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties, dropouts))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty, dropout = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty, dropout)
    score, model = run_experiment(n_hidden_layers = n_hidden_layers,
                                  hidden_layer_size = layer_size,
                                  reg_penalty = reg_penalty,
                                  dropout = dropout,
                                  epochs=EPOCHS)
    experiments[key] = score 
    modelname = "%s_%.6f_%d_%d_%.6f_%.3f" % (MODELPREFIX, score, n_hidden_layers, layer_size, reg_penalty, dropout)
    print("%s Saving %s.h5" % (time.strftime("%H:%M:%S"), modelname))
    model.save("%s.h5" % modelname)
    model.save_weights("%s_weights.h5" % modelname)
    

00:19:49 Running 27 experiments
00:19:49 Running experiment 1 of 27
00:19:49 Generate splits [598, 1196, 1793, 2390, 2987]
00:19:49 Build model
layer 0 size 4, reg_penalty 0.00000000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 4)                 100       
_________________________________________________________________
Dropout00 (Dropout)          (None, 4)                 0         
_________________________________________________________________
output00 (Dense)             (None, 1)                 5         
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________
None
Compile time : 0.35237669944763184
Starting to train : 00:19:49
Training 

00:20:32 Running experiment 6 of 27
00:20:32 Generate splits [598, 1196, 1793, 2390, 2987]
00:20:32 Build model
layer 0 size 8, reg_penalty 0.01000000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 8)                 200       
_________________________________________________________________
Dropout00 (Dropout)          (None, 8)                 0         
_________________________________________________________________
output00 (Dense)             (None, 1)                 9         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________
None
Compile time : 0.7356359958648682
Starting to train : 00:20:33
Training indexes 0 to 1195
Cross-validatin

..Stopping early

00:21:15 Best Xval loss epoch 0, value 0.873484
Training indexes 0 to 2389
Cross-validating indexes 2390 to 2986
......Stopping early

00:21:17 Best Xval loss epoch 3, value 1.358259
Last Xval loss 1.358259
Avg Xval loss 0.989867
--------------------------------------------------------------------------------
00:21:17 Saving FFNN_0.989867_2_4_0.000000_0.250.h5
00:21:18 Running experiment 11 of 27
00:21:18 Generate splits [598, 1196, 1793, 2390, 2987]
00:21:18 Build model
layer 0 size 4, reg_penalty 0.00100000, dropout 0.250
layer 1 size 4, reg_penalty 0.00100000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 4)                 100       
_________________________________________________________________
Dr

..........Stopping early

00:22:35 Best Xval loss epoch 8, value 1.130978
Last Xval loss 1.130978
Avg Xval loss 1.000881
--------------------------------------------------------------------------------
00:22:35 Saving FFNN_1.000881_2_8_0.001000_0.250.h5
00:22:37 Running experiment 15 of 27
00:22:37 Generate splits [598, 1196, 1793, 2390, 2987]
00:22:37 Build model
layer 0 size 8, reg_penalty 0.01000000, dropout 0.250
layer 1 size 8, reg_penalty 0.01000000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 8)                 200       
_________________________________________________________________
Dropout00 (Dropout)          (None, 8)                 0         
_______________________________________________________________

00:23:24 Running experiment 19 of 27
00:23:24 Generate splits [598, 1196, 1793, 2390, 2987]
00:23:24 Build model
layer 0 size 4, reg_penalty 0.00000000, dropout 0.250
layer 1 size 4, reg_penalty 0.00000000, dropout 0.250
layer 2 size 4, reg_penalty 0.00000000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 4)                 100       
_________________________________________________________________
Dropout00 (Dropout)          (None, 4)                 0         
_________________________________________________________________
Dense01 (Dense)              (None, 4)                 20        
_________________________________________________________________
Dropout01 (Dropout)          (None, 4)                 0         

Compile time : 0.23601770401000977
Starting to train : 00:25:58
Training indexes 0 to 1195
Cross-validating indexes 1196 to 1792
................Stopping early

00:26:10 Best Xval loss epoch 14, value 0.662657
Training indexes 0 to 1792
Cross-validating indexes 1793 to 2389
.....Stopping early

00:26:12 Best Xval loss epoch 3, value 0.700423
Training indexes 0 to 2389
Cross-validating indexes 2390 to 2986
......Stopping early

00:26:14 Best Xval loss epoch 3, value 1.216478
Last Xval loss 1.216478
Avg Xval loss 0.859852
--------------------------------------------------------------------------------
00:26:14 Saving FFNN_0.859852_3_8_0.000000_0.250.h5
00:26:17 Running experiment 23 of 27
00:26:17 Generate splits [598, 1196, 1793, 2390, 2987]
00:26:17 Build model
layer 0 size 8, reg_penalty 0.00100000, dropout 0.250
layer 1 size 8, reg_penalty 0.00100000, dropout 0.250
layer 2 size 8, reg_penalty 0.00100000, dropout 0.250
_________________________________________________________________


Compile time : 0.471435546875
Starting to train : 00:27:17
Training indexes 0 to 1195
Cross-validating indexes 1196 to 1792
...Stopping early

00:27:26 Best Xval loss epoch 1, value 0.518482
Training indexes 0 to 1792
Cross-validating indexes 1793 to 2389
....Stopping early

00:27:28 Best Xval loss epoch 2, value 0.639613
Training indexes 0 to 2389
Cross-validating indexes 2390 to 2986
.....Stopping early

00:27:30 Best Xval loss epoch 3, value 1.416213
Last Xval loss 1.416213
Avg Xval loss 0.858103
--------------------------------------------------------------------------------
00:27:30 Saving FFNN_0.858103_3_16_0.001000_0.250.h5
00:27:32 Running experiment 27 of 27
00:27:32 Generate splits [598, 1196, 1793, 2390, 2987]
00:27:32 Build model
layer 0 size 16, reg_penalty 0.01000000, dropout 0.250
layer 1 size 16, reg_penalty 0.01000000, dropout 0.250
layer 2 size 16, reg_penalty 0.01000000, dropout 0.250
_________________________________________________________________
Layer (type)     

In [45]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]

lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "dropout",
                                            "loss"])
lossframe.sort_values(['loss'])

,n_hidden_layers,layer_size,reg_penalty,dropout,loss
7,1,16,0.001,0.25,0.121647
8,1,16,0.010,0.25,0.129034
6,1,16,0.000,0.25,0.209762
5,1,8,0.010,0.25,0.269226
3,1,8,0.000,0.25,0.284692
4,1,8,0.001,0.25,0.320903
1,1,4,0.001,0.25,0.337704
2,1,4,0.010,0.25,0.434735
15,2,16,0.000,0.25,0.543191
14,2,8,0.010,0.25,0.589651


In [46]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
# if a more parsimonious model has nearly same result, pick more parsimonious model
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


,loss
n_hidden_layers,
1,0.341488
2,0.901725
3,1.497414


In [47]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())


,loss
layer_size,
4,1.187875
8,0.883503
16,0.669250


In [48]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())


,loss
reg_penalty,
0.000,0.945283
0.001,0.902221
0.010,0.893123


In [49]:
pd.DataFrame(lossframe.groupby(['dropout'])['loss'].mean())


,loss
dropout,
0.25,0.913542


In [50]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[x_labels], columns=[y_labels], values=['loss'])
    # specify labels as strings, to force it to use a discrete axis
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" units", y_suffix=" layers")



['4  units', '8  units', '16  units'] ['1  layers', '2  layers', '3  layers']


In [51]:
plot_matrix(lossframe, "reg_penalty", "dropout", x_suffix=" d", y_suffix=" r")


['0.250000  d'] ['0.000000  r', '0.001000  r', '0.010000  r']


In [23]:
# rerun best (or load from file)
score, model = run_experiment(n_hidden_layers = 1,
                              hidden_layer_size = 16,
                              reg_penalty = 0.01,
                              dropout = 0.25,
                              epochs=500)

00:48:28 Generate splits [598, 1196, 1793, 2390, 2987]
00:48:28 Build model
layer 0 size 16, reg_penalty 0.01000000, dropout 0.250
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 24)                0         
_________________________________________________________________
Dense00 (Dense)              (None, 16)                400       
_________________________________________________________________
Dropout00 (Dropout)          (None, 16)                0         
_________________________________________________________________
output00 (Dense)             (None, 1)                 17        
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________
None
Compile time : 0.29652976989746094
Starting to train : 00:48:29
Training indexes 0 to 1195
Cross-validating indexes 1196 to 1792
.....Stoppi

In [24]:
z = model.predict(X)
z

array([[-0.5890208],
       [-0.6685476],
       [-0.6657933],
       ...,
       [ 5.0997157],
       [ 5.242778 ],
       [ 4.988749 ]], dtype=float32)

In [25]:
mychart(np.array(range(z.shape[0])), Y, z)


In [26]:
mychart(np.array(range(z.shape[0])), Y-z)
# error grows in test period, doesn't fully capture trend
# should do walk forward, train on 1000, predict 100, train on 1100, etc.